In [5]:
import pandas as pd
import sys
from catboost import CatBoostClassifier 
import numpy as np
from catboost import Pool, cv

In [4]:
train  = pd.read_excel('train.xlsx')
test = pd.read_excel('test.xlsx')
result = pd.read_excel('result.xlsx')

In [17]:
prices = {'bk': 2450, 'GZ1': 2050, 'GZ2': 2050, 'GZ3': 2050,
          'GZ4': 2050, 'GZ5': 2050, 'GZ7': 2050,
          'DGK': 1300, 'NKTD': 2050, 'NKTM': 2050, 'NKTR': 2050,
          'ALPS': 1150}

train = train[train['well id'].notnull()]
train.head()

,well id,"depth, m",bk,GZ1,GZ2,GZ3,GZ4,GZ5,GZ7,DGK,NKTD,NKTM,NKTR,ALPS,lith,goal
0,12.0,3042.4630,0.455779,0.774000,0.774000,0.332846,0.476545,0.774000,0.244898,0.123651,0.774000,0.799167,0.174639,0.727789,NK,0.0
1,33.0,2412.8305,NaN,0.157153,0.755472,0.453216,0.715147,NaN,0.523432,0.388729,0.519233,0.799167,0.563573,0.111216,ARGILLIT,0.0
2,33.0,2522.5575,NaN,0.172235,0.941742,0.525711,0.668600,NaN,0.588924,0.515214,0.527139,0.799167,0.551152,0.172359,ARGILLIT,0.0
3,49.0,2396.3625,0.381248,0.246838,0.116913,0.732654,0.151299,0.165219,0.656955,0.445917,0.575175,0.799167,0.429433,0.253563,ARGILLIT,0.0
4,49.0,2399.0475,0.356867,0.226241,0.155473,0.613300,0.173949,0.297194,0.773218,0.392743,0.575725,0.799167,0.447169,0.959633,ARGILLIT,0.0


In [7]:
%%time
train_many_features = pd.read_csv('fixed_train.csv')

CPU times: user 8.18 s, sys: 644 ms, total: 8.82 s
Wall time: 8.82 s


In [13]:
train = train[train['well id'].notnull()]

In [15]:
train.fillna(0, inplace=True)

/home/iprovilkov/data/p3.5/lib/python3.5/site-packages/pandas/core/frame.py:4034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [17]:
test.fillna(0, inplace=True)

In [18]:
from sklearn import preprocessing

In [56]:
dense_train = train[train.columns[1:-2]]

In [57]:
min_max_scaler = preprocessing.MinMaxScaler()
dense_train_scaled = min_max_scaler.fit_transform(dense_train)

In [58]:
dense_train_scaled[:5]

array([[0.9482561 , 0.45584762, 0.77402056, 0.77418891, 0.33284805,
        0.47656398, 0.77490357, 0.24498382, 0.12371135, 0.96798537,
        1.        , 0.17488977, 0.72779162],
       [0.66026691, 0.        , 0.15715676, 0.7556562 , 0.45321847,
        0.71517533, 0.        , 0.52361605, 0.38891753, 0.64936715,
        1.        , 0.56438115, 0.11121639],
       [0.71045522, 0.        , 0.17223992, 0.94197218, 0.52571448,
        0.66862663, 0.        , 0.58913154, 0.51546392, 0.65925427,
        1.        , 0.55194216, 0.17235956],
       [0.65273457, 0.3813061 , 0.24684478, 0.11694138, 0.73265874,
        0.1513046 , 0.16541176, 0.65718639, 0.44613402, 0.71932995,
        1.        , 0.43004867, 0.25356386],
       [0.65396267, 0.35692142, 0.22624735, 0.15551119, 0.61330346,
        0.17395564, 0.29754131, 0.77348978, 0.39293354, 0.72001775,
        1.        , 0.44780955, 0.9596364 ]])

In [59]:
target = train['goal']

In [60]:
import random
import numpy as np

In [61]:
a = np.arange(target.shape[0])

In [62]:
from sklearn.model_selection import train_test_split

In [63]:
default = ['depth']
bk = ['bk']
GZ = ['GZ1', 'GZ2', 'GZ3', 'GZ4', 'GZ5', 'GZ7']
DGK = ['DGK']
NKT = ['NKTD', 'NKTM', 'NKTR']
ALPS = ['ALPS']

In [64]:
all_f = [bk, GZ, DGK, NKT,ALPS]

In [65]:
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [66]:
from sklearn.linear_model import SGDClassifier, LogisticRegression, Perceptron, PassiveAggressiveClassifier

In [67]:
from sklearn.metrics import f1_score

In [68]:
dense_train_scaled[:5]

array([[0.9482561 , 0.45584762, 0.77402056, 0.77418891, 0.33284805,
        0.47656398, 0.77490357, 0.24498382, 0.12371135, 0.96798537,
        1.        , 0.17488977, 0.72779162],
       [0.66026691, 0.        , 0.15715676, 0.7556562 , 0.45321847,
        0.71517533, 0.        , 0.52361605, 0.38891753, 0.64936715,
        1.        , 0.56438115, 0.11121639],
       [0.71045522, 0.        , 0.17223992, 0.94197218, 0.52571448,
        0.66862663, 0.        , 0.58913154, 0.51546392, 0.65925427,
        1.        , 0.55194216, 0.17235956],
       [0.65273457, 0.3813061 , 0.24684478, 0.11694138, 0.73265874,
        0.1513046 , 0.16541176, 0.65718639, 0.44613402, 0.71932995,
        1.        , 0.43004867, 0.25356386],
       [0.65396267, 0.35692142, 0.22624735, 0.15551119, 0.61330346,
        0.17395564, 0.29754131, 0.77348978, 0.39293354, 0.72001775,
        1.        , 0.44780955, 0.9596364 ]])

In [69]:
str_to_id = {}

for i, st in enumerate(list(dense_train.columns)):
    str_to_id[st] = i

In [70]:
str_to_id

{'ALPS': 12,
 'DGK': 8,
 'GZ1': 2,
 'GZ2': 3,
 'GZ3': 4,
 'GZ4': 5,
 'GZ5': 6,
 'GZ7': 7,
 'NKTD': 9,
 'NKTM': 10,
 'NKTR': 11,
 'bk': 1,
 'depth, m': 0}

In [88]:
dense_train_scaled.shape

(463897, 13)

In [89]:
features_list = []
scores = []
for subset in powerset(all_f):
    features_list.append(subset)
    cls = Perceptron()
    lst = []
    for i in subset:
        lst += i
    mask = [str_to_id[st] for st in lst]
    if len(mask) == 0:
        continue
    X_train, X_test, y_train, y_test = train_test_split(dense_train_scaled[:, mask], target, test_size=0.33, random_state=42)
    cls.fit(X_train, y_train)
    pred = cls.predict(X_test)
    score = f1_score(y_test, pred)
    print(score)
    scores.append(score)

/home/iprovilkov/data/p3.5/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in Perceptron in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/home/iprovilkov/data/p3.5/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.0


/home/iprovilkov/data/p3.5/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in Perceptron in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


0.2956578164710972


/home/iprovilkov/data/p3.5/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in Perceptron in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/home/iprovilkov/data/p3.5/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


...

0.0083500334001336


/home/iprovilkov/data/p3.5/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in Perceptron in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


0.06802113567016448


/home/iprovilkov/data/p3.5/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in Perceptron in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


0.1475071556946016


In [90]:
idx = np.argsort(scores)[::-1]

In [91]:
scores

[0.0,
 0.2956578164710972,
 0.0,
 0.18498664022720016,
 0.0,
 0.19888049635824118,
 0.03275920297196893,
 0.0,
 0.0,
 0.29681378730559066,
 0.28206008365434737,
 0.24938738360288457,
 0.01976748896180176,
 0.00016655562958027982,
 0.0628519124622084,
 0.037019075331393474,
 0.0018956529231829735,
 0.09876890608512136,
 0.09145910269505775,
 0.0,
 0.0,
 0.27403966410577096,
 0.1012718271026652,
 0.1031802650846694,
 0.3080499775671965,
 0.3074725200847087,
 0.16316989071357604,
 0.3440744483967081,
 0.0083500334001336,
 0.06802113567016448,
 0.1475071556946016]

In [92]:
features_list

[(),
 (['bk'],),
 (['GZ1', 'GZ2', 'GZ3', 'GZ4', 'GZ5', 'GZ7'],),
 (['DGK'],),
 (['NKTD', 'NKTM', 'NKTR'],),
 (['ALPS'],),
 (['bk'], ['GZ1', 'GZ2', 'GZ3', 'GZ4', 'GZ5', 'GZ7']),
 (['bk'], ['DGK']),
 (['bk'], ['NKTD', 'NKTM', 'NKTR']),
 (['bk'], ['ALPS']),
 (['GZ1', 'GZ2', 'GZ3', 'GZ4', 'GZ5', 'GZ7'], ['DGK']),
 (['GZ1', 'GZ2', 'GZ3', 'GZ4', 'GZ5', 'GZ7'], ['NKTD', 'NKTM', 'NKTR']),
 (['GZ1', 'GZ2', 'GZ3', 'GZ4', 'GZ5', 'GZ7'], ['ALPS']),
 (['DGK'], ['NKTD', 'NKTM', 'NKTR']),
 (['DGK'], ['ALPS']),
 (['NKTD', 'NKTM', 'NKTR'], ['ALPS']),
 (['bk'], ['GZ1', 'GZ2', 'GZ3', 'GZ4', 'GZ5', 'GZ7'], ['DGK']),
 (['bk'],
  ['GZ1', 'GZ2', 'GZ3', 'GZ4', 'GZ5', 'GZ7'],
  ['NKTD', 'NKTM', 'NKTR']),
 (['bk'], ['GZ1', 'GZ2', 'GZ3', 'GZ4', 'GZ5', 'GZ7'], ['ALPS']),
 (['bk'], ['DGK'], ['NKTD', 'NKTM', 'NKTR']),
 (['bk'], ['DGK'], ['ALPS']),
 (['bk'], ['NKTD', 'NKTM', 'NKTR'], ['ALPS']),
 (['GZ1', 'GZ2', 'GZ3', 'GZ4', 'GZ5', 'GZ7'],
  ['DGK'],
  ['NKTD', 'NKTM', 'NKTR']),
 (['GZ1', 'GZ2', 'GZ3', 'GZ4', 'GZ5',

In [93]:
for i in idx[:10]:
    print(features_list[i])

(['bk'], ['GZ1', 'GZ2', 'GZ3', 'GZ4', 'GZ5', 'GZ7'], ['DGK'], ['ALPS'])
(['GZ1', 'GZ2', 'GZ3', 'GZ4', 'GZ5', 'GZ7'], ['NKTD', 'NKTM', 'NKTR'], ['ALPS'])
(['DGK'], ['NKTD', 'NKTM', 'NKTR'], ['ALPS'])
(['bk'], ['ALPS'])
(['bk'],)
(['GZ1', 'GZ2', 'GZ3', 'GZ4', 'GZ5', 'GZ7'], ['DGK'])
(['bk'], ['NKTD', 'NKTM', 'NKTR'], ['ALPS'])
(['GZ1', 'GZ2', 'GZ3', 'GZ4', 'GZ5', 'GZ7'], ['NKTD', 'NKTM', 'NKTR'])
(['ALPS'],)
(['DGK'],)


In [98]:
sorted(scores)[::-1]

[0.3440744483967081,
 0.3080499775671965,
 0.3074725200847087,
 0.29681378730559066,
 0.2956578164710972,
 0.28206008365434737,
 0.27403966410577096,
 0.24938738360288457,
 0.19888049635824118,
 0.18498664022720016,
 0.16316989071357604,
 0.1475071556946016,
 0.1031802650846694,
 0.1012718271026652,
 0.09876890608512136,
 0.09145910269505775,
 0.06802113567016448,
 0.0628519124622084,
 0.037019075331393474,
 0.03275920297196893,
 0.01976748896180176,
 0.0083500334001336,
 0.0018956529231829735,
 0.00016655562958027982,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]